In [2]:
import pandas as pd
from tsnn.data_utils import *
from tsnn.models import DeepRecurrent, LSTNet, DeepSense

In [3]:
# Prediction task
input_cols = []
target_cols = ["client_0", "client_6", "client_11", "client_36", "client_315"]
pred_delay = 24

# Hyperparameters
scaling_method = "maxabs"
timesteps = 168
batch_size = 64
sampling_step = 1

# Simple cross-validation
train_ratio = 0.6
val_ratio = 0.2

In [4]:
# Load data
raw_data = pd.read_csv('~/Desktop/electricity.txt', header=None)
raw_data.columns = ["client_" + str(i) for i in range(len(raw_data.columns))] 
nb_features = max(len(input_cols), len(raw_data.columns))

# Converting column names to their indices (necessary for LSTNet only)
target_cols_indices = colnames_to_colindices(interest_cols=target_cols, original_df=raw_data)

# Prepare data generators
generators_dict, stats_df = prepare_data_generators(raw_data=raw_data, 
                                                    input_cols=input_cols, 
                                                    target_cols=target_cols,     # We use the column NAMES here
                                                    scaling_method="maxabs", 
                                                    samples_length=timesteps, 
                                                    pred_delay=pred_delay,
                                                    sampling_step=sampling_step, 
                                                    batch_size=batch_size, 
                                                    train_ratio=train_ratio, 
                                                    val_ratio=val_ratio)

train_gen, train_gen_steps = generators_dict["train"]
val_gen, val_gen_steps = generators_dict["val"]
test_gen, test_gen_steps = generators_dict["test"]

# LSTNet

In [5]:
# Create model
lstnet = LSTNet(input_shape=(timesteps, nb_features), 
                interest_vars=target_cols_indices,     # We use the column INDICES here
                cnn_filters=100, 
                cnn_kernel_height=6, 
                cnn_activation='relu', 
                cnn_use_bias=True,
                gru_units=100, 
                gru_activation='relu', 
                gru_use_bias=True,
                gru_skip_units=5, 
                gru_skip_step=24, 
                gru_skip_activation='relu', 
                gru_skip_use_bias=True,
                ar_window=24, 
                ar_use_bias=True, 
                dropout=0.2)
lstnet.compile(loss='mae', optimizer='adam')

In [5]:
# Train the model
lstnet.fit_generator(generator=train_gen, 
                     steps_per_epoch=train_gen_steps, 
                     validation_data=val_gen, 
                     validation_steps=val_gen_steps,
                     epochs=3, 
                     shuffle=False)

Epoch 1/3
245/245 [==============================] - 155s - loss: 0.1470 - val_loss: 0.1149
Epoch 2/3
245/245 [==============================] - 154s - loss: 0.1069 - val_loss: 0.1023
Epoch 3/3
245/245 [==============================] - 154s - loss: 0.0978 - val_loss: 0.0946


In [7]:
# Predict on test set
pred_test_gen = yield_inputs_only(test_gen)
lstnet_preds = lstnet.predict_generator(generator=pred_test_gen, steps=test_gen_steps)

# DeepRecurrent

In [13]:
# Create model
deeprec = DeepRecurrent(input_shape=(timesteps,nb_features),
                        rec_layers_dims=[128,64], 
                        rec_layer_type="lstm", 
                        rec_use_bias=True, 
                        rec_activation='tanh',
                        output_dim=5, 
                        dense_use_bias=True, 
                        dropout=0.2)
deeprec.compile(loss='mae', optimizer='adam')
deeprec.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Main_input (InputLayer)      (None, 168, 321)          0         
_________________________________________________________________
Recurrent_1 (GRU)            (None, 168, 128)          172800    
_________________________________________________________________
Recurrent_2 (GRU)            (None, 64)                37056     
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 325       
Total params: 210,181
Trainable params: 210,181
Non-trainable params: 0
_________________________________________________________________


In [14]:
deeprec.fit_generator(generator=train_gen, 
                     steps_per_epoch=train_gen_steps, 
                     validation_data=val_gen, 
                     validation_steps=val_gen_steps,
                     epochs=3, 
                     shuffle=False)

Epoch 1/3
 19/245 [=>............................] - ETA: 184s - loss: 0.1532

KeyboardInterrupt: 